In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import zipfile
with zipfile.ZipFile('../input/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

In [16]:
data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

In [17]:
#remove all files .DS_Store
for parent, dirnames, filenames in os.walk(data_root):
    for fn in filenames:
        if fn.lower().endswith('.ds_store'):
            os.remove(os.path.join(parent, fn))

In [18]:
# The shutil module offers a number of high-level operations on files and collections of files. 
# In particular, functions are provided which support file copying and removal.
import shutil
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

#create dirs ./train/cleaned, /train/dirty, ./val/cleaned, /val/dirty,
for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

#fill directories train (5/6)=83% and val (1/6)=17%, 
for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [19]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import matplotlib
import time
import copy
from datetime import datetime

from torchvision import transforms, models

# rs = 0
# random.seed(rs)
# np.random.seed(rs)
# torch.manual_seed(rs)
# torch.cuda.manual_seed(rs)
# torch.backends.cudnn.deterministic = True

In [20]:
# Transforms are common image transformations. They can be chained together using Compose. 
# Most transform classes have a function equivalent: functional transforms give fine-grained 
# control over the transformations. This is useful if you have to build a more complex transformation 
# pipeline (e.g. in the case of segmentation tasks).

# ILLUSTRATION OF TRANSFORMS: https://pytorch.org/vision/stable/auto_examples/plot_transforms.html#sphx-glr-auto-examples-plot-transforms-py

train_transforms = transforms.Compose([
    #!!очень спорное применение RandomResizedCrop для нашего случая, 
    #т.к. по итогу абсолютно получается рандомно вырезанный, сжатый/растянутый кусок исходного изображения
    transforms.RandomResizedCrop(224), 
    transforms.RandomHorizontalFlip(), #с вероятностью 50% отражаем по горизонтали
    transforms.ToTensor(),
    #нормализуем в соостветсвии с тем, как, были предобработаны изображения imagenet1000, при обучении resnet
    # mean = [0.485, 0.456, 0.406], эти значения вычитаются из RGB каналов изображения (т.е. нормализуем смещение)
    # std = [0.229, 0.224, 0.225], на эти значения делим RGB каналы изображения 
    #      (т.е. нормализцем среднеквадратичное (стандартное) отклонение, std^2 = var)
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)), #важно, не квадратные изображения будут сжиматься/растягиваться!
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


#получаем объект типа .ImageFolder, для дальнейшего использования через torch.utils.data.DataLoader
#по сути связывает директорию с изображениями, разбитыми ко поддиректориям=классам, с трансформациями, 
#которые будут производиться при дальнейшей загрузке изображений в память (в том числе в композиции 
#трансформаций выше сразу зашито преобразование в тензор)
#Важные особенности:
# - указанная директория обязательно должна содержать поддиректории соответсвующие названиям классов, 
#   уже в свою очередь в которых должны находиться изображения
# - по итогу, мы получаем объект типа ImageFolder, который на первый взгляд является последовательностью 
#   (sequential), длины = кол-ву изображений во всех поддиректориях, но по факту, является только ссылками
#   на изображения, т.е. в память сразу они не загружаются, а только в момент обращения к конкретному
#   объекту; простая проверка, если удалить изображение(-я) с диска, заранее созданный объект ImageFolder
#   уже не сможет предоставить доступ к соответсвующему удаленному изображению объекту (с исключением о 
#   невозможности найти файл)
# - важно понимать!, что каждый раз при обращении к конкретному элементу (например с индексом 0), весь 
#   процесс загрузки с диска и применение трансформаций будет запускаться заново, как следствие, если мы
#   имеем рандомные трансформации, т.е. каждый раз при обращении к одному и тому же объекту мы будем 
#   получать разные результаты
# - для нашего случая трансформаций (где присутсвует .ToTensor()), мы получим последовательность кортежей, 
#   из 2-х подэлементов, 0-й элемент = тензору соответсвующему изображению, 1-й элемент целое число, 
#   отнесение к классу (насколько я понял, классы назначаются в порядке алфавитной сортировки поддиректорий, 
#   т.е. для нашего случая cleaned = 0, dirty = 1)

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)
len(train_dataset), len(val_dataset)

In [21]:
# На основе .ImageFolder генерируем подобие итератора/последовательности псевдо-батчей указанного размера 
# (в зависимости от shuffle, перемешиваем их или нет), псевдо-, потому что данные не загружаются сразу, 
# а только в момет обращения к ним, для этого указывается количество воркеров (для многопоточности).
# Важные примечания:
# - с учетом нашей реализации .ImageFolder, где происходят рандомные трансформации, каждый раз получем
#   рандомно измененные изображения, даже для одинаковых батчей
# - изображения в каждом батче, так же перемешиваются каждый раз при вызове запросе батча
# - это все же, как понял это некоторый микс итератора/последовательности, т.к. например, у объекта есть 
#   размер len(dataloader), но при этом обратиться к индексу нельзя, просто вызвать next(dataloader)
#   нельзя, зато можно вызвать предварительно приведя к классическому итератору через next(iter(dataloader))
# - при получении следующего батча (пример выше), возвращает 2 объекта/датасета с фичами (X) и labels (y)
#   (для нашего случая возвращает 2 тензора, каждый батч фичей будет torch.Size([8, 3, 224, 224]), батч 
#   лейблов torch.Size([8]))

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [22]:
len(train_dataloader), len(val_dataloader)

In [23]:
# X_batch, y_batch = next(iter(train_dataloader))
# mean = np.array([0.485, 0.456, 0.406])
# std = np.array([0.229, 0.224, 0.225])
# #permute(1, 2, 0) необходим, т.к. для нескольких каналов, plt принимает тензор, 
# #где измерение (dim) каналов на последнем месте, а в торче каналы стоят перед 
# #размерами изображения (на 2 месте)
# #так же производим денормализацию (там делили вычитали, здесь умножаем складываем), 
# #противоположную той, которую произвели в трансформере во время загрузки изображения
# plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean); 

In [24]:
def show_input(input_tensor, title='', 
               mean = np.array([0.485, 0.456, 0.406]), 
               std = np.array([0.229, 0.224, 0.225])):
    '''Функция реализующая вывод изображений с заголовком = классу'''
    
    #permute(1, 2, 0) необходим, т.к. для нескольких каналов, plt принимает тензор, 
    #где измерение (dim) каналов на последнем месте, а в торче каналы стоят перед 
    #размерами изображения (на 2 месте)
    #так же производим денормализацию (там делили вычитали, здесь умножаем складываем), 
    #противоположную той, которую произвели в трансформере во время загрузки изображения
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1)) #.clip загоняет все значения в заданный интервал, т.е. [0, 1]
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [37]:
def show_experiments_plots(accuracies, losses, figsize = (16.0, 6.0)):
    matplotlib.rcParams['figure.figsize'] = figsize
    
    for experiment_id in accuracies.keys():
        print('{:-<100}'.format(experiment_id))
        epoch_max_acc = np.array(accuracies[experiment_id]['val']).argmax()
        print('Max val accuracy on: Epoch = {:>3},     ACCURACY: val = {:.3f}, train = {:.3f},     LOSS: val = {:.3f}, train = {:.3f}'\
              .format(epoch_max_acc, 
                      accuracies[experiment_id]['val'][epoch_max_acc], 
                      accuracies[experiment_id]['train'][epoch_max_acc],
                      losses[experiment_id]['val'][epoch_max_acc],
                      losses[experiment_id]['train'][epoch_max_acc]))
        epoch_min_loss = np.array(losses[experiment_id]['val']).argmin()
        print('Min val loss on:     Epoch = {:>3},     ACCURACY: val = {:.3f}, train = {:.3f},     LOSS: val = {:.3f}, train = {:.3f}'\
              .format(epoch_min_loss, 
                      accuracies[experiment_id]['val'][epoch_min_loss], 
                      accuracies[experiment_id]['train'][epoch_min_loss],
                      losses[experiment_id]['val'][epoch_min_loss],
                      losses[experiment_id]['train'][epoch_min_loss]))
    
    for experiment_id in accuracies.keys():
        plt.plot(accuracies[experiment_id]['val'], label=experiment_id + ' val')
    plt.legend()
    plt.title('Validation Accuracy (Val only)')
    plt.show()

    for experiment_id in accuracies.keys():
        plt.plot(accuracies[experiment_id]['val'], label=experiment_id + ' val')
        plt.plot(accuracies[experiment_id]['train'], label=experiment_id + ' train')
    plt.legend()
    plt.title('Validation Accuracy (Val/Train)');
    plt.show()

    for experiment_id in losses.keys():
        plt.plot(losses[experiment_id]['val'], label=experiment_id  + ' val')
    plt.legend()
    plt.title('Validation Loss (Val only)');
    plt.show()

    for experiment_id in losses.keys():
        plt.plot(losses[experiment_id]['val'], label=experiment_id  + ' val')
        plt.plot(losses[experiment_id]['train'], label=experiment_id  + ' train')
    plt.legend()
    plt.title('Validation Loss (Val/Train)');
    plt.show()

In [30]:
def train_model(model, loss, optimizer, scheduler, num_epochs, with_validation = True):
    
    accuracy_history = {}
    loss_history = {}
    accuracy_history['train'] = []
    loss_history['train'] = []
    accuracy_history['val'] = []
    loss_history['val'] = []
    
    for epoch in range(num_epochs):
        
        #####train phase######
        train_accuracy_epoch = [] #for statistics
        train_loss_epoch = [] #for statistics
        
        model.train()  # Set model to training mode
        scheduler.step() #no grad step, only change lr of optimizer

        for inputs, labels in train_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                preds = model(inputs) #аналогично вызову метода model.forward(inputs)
                loss_value = loss(preds, labels)
                
                loss_value.backward()
                optimizer.step()
                
                train_accuracy_epoch.append(float((preds.argmax(dim=1) == labels.data).float().mean().data))
                #.item() Returns the value of this tensor as a standard Python number. This only works for tensors with one element.
                train_loss_epoch.append(loss_value.item())
                
        #####validation phase######
        val_accuracy_epoch = []
        val_loss_epoch = []
        
        model.eval()   #Set model to evaluate mode
        
        for inputs, labels in val_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
        
                with torch.no_grad(): #не храним градиенты для теста, многоркатно сокращает потребление памяти                  
                    preds = model(inputs) #аналогично вызову метода model.forward(inputs)
                    loss_value = loss(preds, labels)
                
                    val_accuracy_epoch.append(float((preds.argmax(dim=1) == labels.data).float().mean().data))
                    val_loss_epoch.append(loss_value.item())
                    
        ########ending of epoch#########
        accuracy_history['val'].append(sum(val_accuracy_epoch) / len(val_accuracy_epoch))
        loss_history['val'].append(sum(val_loss_epoch) / len(val_loss_epoch))
        accuracy_history['train'].append(sum(train_accuracy_epoch) / len(train_accuracy_epoch))
        loss_history['train'].append(sum(train_loss_epoch) / len(train_loss_epoch))    
        
        print('Epoch = {:>3}/{},     ACCURACY: val = {:.3f}, train = {:.3f},     LOSS: val = {:.3f}, train = {:.3f}'\
                  .format(epoch,
                          num_epochs - 1,
                          accuracy_history['val'][-1], 
                          accuracy_history['train'][-1],
                          loss_history['val'][-1],
                          loss_history['train'][-1]), 
                  flush=True)

    return model, accuracy_history, loss_history

In [27]:
accuracies = {}
losses = {}

In [28]:
model = models.resnet18(pretrained=True) #загружаем предтренированную на imagenet1000 модель

# Disable grad for all conv layers (скорей всего морозим градиенты не только conv)
for param in model.parameters():
    param.requires_grad = False
    
#заменяем последний полносвязный слой, содержащий 1000 нейронов (для 1000 классов imagenet1000)
#на полносвязный слой из 2-х нейронов, для наших 2-х классов (кстати странно что не используется 
#классическая бинарная конфигурация, с 1-м выходом и BCE, нужно проверить, даст ли замена на 
#классику более лучший результат)
#последний слой хранится в параметре .fc экземпляра класса, до замены он был = 
#Linear(in_features=512, out_features=1000, bias=True)
model.fc = torch.nn.Linear(model.fc.in_features, 2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-3)

# Decay LR by a factor of 0.1 every 7 epochs
# Единственное назначение .lr_scheduler.StepLR, это изменение LR, оптимизатора, путем его домножения 
# на gamma, каждые step_size эпох
# Важно понимать, что scheduler считает эпохи путем явного выхова scheduler.step(),
# а так же, что его .step() не делает ничего более кроме изменения LR каждые step_size эпох т.е. 
# scheduler.step() не отменяет необходимости вызова optimizer.step() для осуществления град. шага

# gamma - гиперпараметр, соответсвенно имеет смысл им поиграть на практике и найти оптимальный
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [31]:
net_name = 'resnet18'
model, accuracies[net_name], losses[net_name] = train_model(model, loss, optimizer, scheduler, num_epochs=100)

In [38]:
show_experiments_plots(accuracies, losses)

In [39]:
test_dir = 'test'
#создаем в test поддиректорию не существующего класса unknown, и копируем туда test изображения
#т.к. .ImageFolder требует, что бы в целевой директории обязательно были поддиректории=классы
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [40]:
#создаем класс ImageFolderWithPaths, на основе ImageFolder, с единственным изменением метода .__getitem__
#что бы кортеж элементов, помимо изображения и класса, содержал еще 3-й элемент = полный путь к изображению
#это необходимо для формирования submit csv, где первый столбец = индекcу фото (имя файла без расширения),
#второй столбец = отнесению к классу
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,)) #сумма кортежей = concat из всех элементов
        return tuple_with_path
    
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [41]:
model.eval() #не забываем перевести модель в eval режим

test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)
    with torch.set_grad_enabled(False):
        preds = model(inputs)
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy())
    test_img_paths.extend(paths) #непонятно почему здесь extend, а не append
    
test_predictions = np.concatenate(test_predictions) #преобразуем list в np.array

In [45]:
inputs, labels, paths = next(iter(test_dataloader))

for img, pred in zip(inputs, test_predictions):
    show_input(img, title=pred)

In [ ]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [ ]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned') #threshold (отсечка) для отнесения к классу 
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)

In [ ]:
submission_df.to_csv('submission_{}.csv'.format(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")))

In [ ]:
!rm -rf train val test plates

In [ ]:
!ls